In [3]:
from cloudpickle import load
from tensorflow.keras.models import model_from_json 
import pandas as pd
import tensorflow as tf

values = [['192.168.60.100', 65, '192.168.40.2',   357,   6, 10, 0, 10, 0, 0, 15360], 
          ['192.168.60.100', 65, '192.168.40.2',   357,   6, 10, 0, 10, 0, 0, 15360],
          ['192.168.50.2'  , 65, '192.168.40.2',   357,   6, 10, 0, 10, 0, 0, 15360],
          ['192.168.50.100', 65, '192.168.60.100', 357,   6, 10, 0, 10, 0, 0, 15360]]

columns = ['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',\
    'PROTOCOL', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS', 'TCP_FLAGS','FLOW_DURATION_MILLISECONDS']
df = pd.DataFrame(values, columns=columns)
#print(df)


def  predict_packets(df):
    result = ['is an anomaly', 'is a normal activity']
    #Import process
    process = load(open('/kaggle/input/detection-anomaly-model/processing.pkl','rb'))
 
    #Import model 
    #from tensorflow.keras.models  import model_from_json

    # load json and create model
    json_file = open('/kaggle/input/detection-anomaly-model/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)

    # load weights into new model
    model.load_weights('/kaggle/input/detection-anomaly-model/model.weights.h5')
    print("Loaded model from disk") 

    columns = ['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',\
    'PROTOCOL', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS',  'OUT_PKTS', 'TCP_FLAGS','FLOW_DURATION_MILLISECONDS']
    df = pd.DataFrame(values, columns=columns)
    df_result =  pd.DataFrame(values, columns=columns) 
    
    
    #Adress source
    df[['IPV4_SRC_ADDR_network1','IPV4_SRC_ADDR_network2','IPV4_SRC_ADDR_hote1',\
    'IPV4_SRC_ADDR_hote2']]=df['IPV4_SRC_ADDR'].str.split('.', expand=True)
    df = process['target_encoding_src_network1'].merge(df, on='IPV4_SRC_ADDR_network1')
    df = process['target_encoding_src_network2'].merge(df, on='IPV4_SRC_ADDR_network2')
    df = process['target_encoding_src_hote1'].merge(df, on='IPV4_SRC_ADDR_hote1')
    df = process['target_encoding_src_hote2'].merge(df, on='IPV4_SRC_ADDR_hote2')

    #Destinaton adress
    df[['IPV4_DST_ADDR_network1','IPV4_DST_ADDR_network2','IPV4_DST_ADDR_hote1',\
    'IPV4_DST_ADDR_hote2']]= df['IPV4_DST_ADDR'].str.split('.', expand=True)
    df = process['target_encoding_dst_network1'].merge(df, on='IPV4_DST_ADDR_network1')
    df = process['target_encoding_dst_network2'].merge(df, on='IPV4_DST_ADDR_network2')
    df = process['target_encoding_dst_hote1'].merge(df, on='IPV4_DST_ADDR_hote1')
    df = process['target_encoding_dst_hote2'].merge(df, on='IPV4_DST_ADDR_hote2')

    #Deal with src port address
    df = process['target_encoding_l4_src_port'].merge(df, on='L4_SRC_PORT')

    #Deal with dst port address
    df = process['target_encoding_l4_dst_port'].merge(df, on='L4_DST_PORT')

    #Deal with protocol
    df= process['target_encoding_protocol'].merge(df, on='PROTOCOL')
    #Deal with tcp_flags
    df= process['target_encoding_tcp_flags'].merge(df, on='TCP_FLAGS')
    
    #Drop col
    drop_col = ['IPV4_SRC_ADDR', 'IPV4_DST_ADDR',  'TCP_FLAGS', 'PROTOCOL', 
    'L4_DST_PORT', 'L4_SRC_PORT', 'IPV4_DST_ADDR_hote2','IPV4_DST_ADDR_hote1','IPV4_DST_ADDR_network2',
    'IPV4_DST_ADDR_network1', 'IPV4_SRC_ADDR_hote2','IPV4_SRC_ADDR_hote1', 'IPV4_SRC_ADDR_network2',
    'IPV4_SRC_ADDR_network1']
    df = df.drop(columns=drop_col)

    #Standard scaler
    df_scaled = process['stdScaler'].transform(df)

    #Prediction
    reconstruction_result = model.predict(df_scaled)
    loss = tf.keras.losses.mae(reconstruction_result, df_scaled)
   
    result = ['Activité normale', 'Anomalie']
    threshold = process['threshold']
    outliers = loss > threshold
    print(outliers)
    print('\n')
        
    for index, element  in enumerate(outliers):
        a = element.numpy().any().astype(int)
        df_result.loc[index, 'Label'] = a
    print(df_result)
    

predict_packets(values)

Loaded model from disk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
tf.Tensor([ True  True  True  True], shape=(4,), dtype=bool)


    IPV4_SRC_ADDR  L4_SRC_PORT   IPV4_DST_ADDR  L4_DST_PORT  PROTOCOL  \
0  192.168.60.100           65    192.168.40.2          357         6   
1  192.168.60.100           65    192.168.40.2          357         6   
2    192.168.50.2           65    192.168.40.2          357         6   
3  192.168.50.100           65  192.168.60.100          357         6   

   IN_BYTES  OUT_BYTES  IN_PKTS  OUT_PKTS  TCP_FLAGS  \
0        10          0       10         0          0   
1        10          0       10         0          0   
2        10          0       10         0          0   
3        10          0       10         0          0   

   FLOW_DURATION_MILLISECONDS  Label  
0                       15360    1.0  
1                       15360    1.0  
2                       15360    1.0  
3                       15360    1.0  
